In [1]:
import pandas as pd
import numpy as np

import plotly.express as px

import pycaret
print(pycaret.__version__)

3.2.0


In [87]:
df_all_plot = pd.read_csv('df_all_plot.csv')
df_all_plot.tail(3)

,year reset,temp cffill mod,anom c mod,co2 mean
68986,821911.833333,NaN,NaN,NaN
68987,821911.916667,NaN,NaN,NaN
68988,821999.000000,-54.85,NaN,NaN


In [3]:
df_all_plot.dropna(inplace=True)

In [15]:
px.scatter(df_all_plot, x='year reset', y=df_all_plot.drop(columns='year reset').columns)

In [4]:
from pycaret.time_series import TSForecastingExperiment

In [5]:
exp = TSForecastingExperiment()

In [10]:
#df_all_plot['year reset'].index

In [79]:
temp = df_all_plot[['year reset', 'temp cffill mod']]
temp.head(2)

,year reset,temp cffill mod
1939,240331.0,-58.45
2326,262377.0,-60.25


In [80]:
#exp.setup(temp, target='temp cffill mod', fh=3, session_id=123,
#          numeric_imputation_target='ffill'
          
 #         )

# fh forecasting horizon: how much in the future yto forecast

In [8]:
#best = exp.compare_models(n_select=3)

In [17]:
temp_ols = px.scatter(temp, x='year reset', y='temp cffill mod', trendline='ols') 

In [18]:
temp_ols.show()

In [24]:
temp_ols_fit = px.get_trendline_results(temp_ols).iloc[0].values[0]

In [38]:
slope, intercept = temp_ols_fit.params[1], temp_ols_fit.params[0]
slope, intercept

(7.486862663792364e-05, -90.0344219592584)

In [26]:
temp_ols_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.207
Model:                            OLS   Adj. R-squared:                  0.195
Method:                 Least Squares   F-statistic:                     17.44
Date:                Tue, 02 Jan 2024   Prob (F-statistic):           8.75e-05
Time:                        21:10:39   Log-Likelihood:                -284.63
No. Observations:                  69   AIC:                             573.3
Df Residuals:                      67   BIC:                             577.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -90.0344     14.366     -6.267      0.000    -118.708     -61.361
x1          7.487e-05   1.79e-05      4.176      0.000    3.91e-05       0.000
==============================================================================
Omnibus:                       14.684   Durbin-Watson:                   0.126
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               18.336
Skew:                          -1.251   Prob(JB):                     0.000104
Kurtosis:                       2.660   Cond. No.                     6.29e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.29e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [81]:
# y = mx + c
# temp = 7.487e05 * year reset + (-90.0344) 

years = np.array([900000, 1000000, 1200000])
temp_predict = slope * years + intercept
temp_predict

array([-22.65265799, -15.16579532,  -0.19206999])

In [82]:
temp_predict = pd.DataFrame([years, temp_predict]).T
temp_predict.columns = temp.columns
temp_predict['label'] = 'predicted'
temp['label'] = 'observed'

#temp_predict = pd.concat([temp, temp_predict])

In [84]:
temp_predict = pd.concat([temp, temp_predict])

In [60]:
#temp_predict['label'] = np.where(temp_predict['label'].isna(), 'observed', temp_predict['label'])
#temp_predict

In [85]:
temp_predict

,year reset,temp cffill mod,label
1939,240331.0,-58.450000,observed
2326,262377.0,-60.250000,observed
5375,527270.0,-62.650000,observed
9121,694216.0,-51.550000,observed
11175,738505.0,-58.150000,observed
...,...,...,...
68952,821909.0,-20.060000,observed
68964,821910.0,-20.010000,observed
0,900000.0,-22.652658,predicted
1,1000000.0,-15.165795,predicted


In [86]:
px.scatter(temp_predict, x='year reset', y='temp cffill mod', color='label')

In [33]:
temp_ols.show()

In [42]:
temp

array([-22.65265799, -15.16579532,  -0.19206999])